<a href="https://www.kaggle.com/code/anggoletomi/package-box-size-optimization?scriptVersionId=105736454" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

We are an online seller that looking for the most fit box dimension that we will use as shipping box to end customers. The goal is to find the most efficient box size that isn't too big to take up a lot of unused space or too small to fit multiple products. We will use the sales data of the last few months to determine the right box size.

## CREATE BOX DIMENSION LIST

In [6]:
import itertools
import pandas as pd
from tqdm import tqdm
from py3dbp import Packer, Bin, Item

## create list with loop

length = []

for x in range(10,30): # min.size is 10cm, max.size is 40cm
    length.append(x)

width, height = length, length

MAX_WEIGHT = 10 ** 9

In [7]:
list_box = [length,width,height]

box_dimension = list(itertools.product(*list_box))

df_box = pd.DataFrame(box_dimension,dtype=str)

df_box.columns = ['length','width','height']

for fl in df_box.columns:
    df_box[fl] = df_box[fl].astype(float)

df_box['id'] = df_box.index + 1

df_box['box_number'] = "BOX-"
df_box['box_number'] = df_box['box_number'] + df_box['id'].astype(str)

df_box['box_dimension'] = df_box['length'].astype(str) + ' x ' + df_box['width'].astype(str) + ' x ' + df_box['height'].astype(str)

df_box = df_box[['id','box_number','length','width','height','box_dimension']]

box_dimension_df = df_box.copy()

print(f'There are {len(box_dimension)} possibility of box dimension')

There are 8000 possibility of box dimension


In [8]:
box_dimension_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             8000 non-null   int64  
 1   box_number     8000 non-null   object 
 2   length         8000 non-null   float64
 3   width          8000 non-null   float64
 4   height         8000 non-null   float64
 5   box_dimension  8000 non-null   object 
dtypes: float64(3), int64(1), object(2)
memory usage: 375.1+ KB


## GET THE SKU LIST

In [9]:
sku_dimension_df = pd.read_csv("/kaggle/input/uq-retail-order/sku_dimensions.csv")

sku_dimension_df.head(5)

,sku_id,length,width,height,sku_dimension
0,UQ001,14.0,9.0,1.0,14 x 9 x 1
1,UQ002,9.0,8.3,20.5,9 x 8.3 x 20.5
2,UQ003,7.0,7.0,14.0,7 x 7 x 14
3,UQ004,7.0,7.0,15.0,7 x 7 x 15
4,UQ005,9.0,9.0,9.0,9 x 9 x 9


In [10]:
sku_dimension_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sku_id         349 non-null    object 
 1   length         349 non-null    float64
 2   width          349 non-null    float64
 3   height         349 non-null    float64
 4   sku_dimension  349 non-null    object 
dtypes: float64(3), object(2)
memory usage: 13.8+ KB


## GET THE ORDER LIST

In [11]:
order_sku_df = pd.read_csv("/kaggle/input/uq-retail-order/uq_orders.csv")

sku_dimension_df.head(5)

,sku_id,length,width,height,sku_dimension
0,UQ001,14.0,9.0,1.0,14 x 9 x 1
1,UQ002,9.0,8.3,20.5,9 x 8.3 x 20.5
2,UQ003,7.0,7.0,14.0,7 x 7 x 14
3,UQ004,7.0,7.0,15.0,7 x 7 x 15
4,UQ005,9.0,9.0,9.0,9 x 9 x 9


In [12]:
sku_dimension_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   sku_id         349 non-null    object 
 1   length         349 non-null    float64
 2   width          349 non-null    float64
 3   height         349 non-null    float64
 4   sku_dimension  349 non-null    object 
dtypes: float64(3), object(2)
memory usage: 13.8+ KB


## CALCULATE THE BOX SIZE

In [13]:
box_dimension_df['volume'] = box_dimension_df['length'] * box_dimension_df['width'] * box_dimension_df['height']
box_dimension_df.sort_values(['volume', 'id'], inplace=True)
box_dimension_df.drop_duplicates('volume', inplace=True)
box_dimension_df

,id,box_number,length,width,height,box_dimension,volume
0,1,BOX-1,10.0,10.0,10.0,10.0 x 10.0 x 10.0,1000.0
1,2,BOX-2,10.0,10.0,11.0,10.0 x 10.0 x 11.0,1100.0
2,3,BOX-3,10.0,10.0,12.0,10.0 x 10.0 x 12.0,1200.0
21,22,BOX-22,10.0,11.0,11.0,10.0 x 11.0 x 11.0,1210.0
3,4,BOX-4,10.0,10.0,13.0,10.0 x 10.0 x 13.0,1300.0
...,...,...,...,...,...,...,...
7578,7579,BOX-7579,28.0,28.0,28.0,28.0 x 28.0 x 28.0,21952.0
7199,7200,BOX-7200,27.0,29.0,29.0,27.0 x 29.0 x 29.0,22707.0
7579,7580,BOX-7580,28.0,28.0,29.0,28.0 x 28.0 x 29.0,22736.0
7599,7600,BOX-7600,28.0,29.0,29.0,28.0 x 29.0 x 29.0,23548.0


In [14]:
df = order_sku_df.copy()
df = df.merge(sku_dimension_df, how='left', on='sku_id')

df = df.dropna()

df

,order_number,sku_id,quantity,length,width,height,sku_dimension
0,SJK121090605204,UQ166,2,15.5,2.5,4.0,15.5 x 2.5 x 4
1,SJK121050600283,UQ002,1,9.0,8.3,20.5,9 x 8.3 x 20.5
2,SJK121072610089,UQ114,1,9.0,6.0,23.0,9 x 6 x 23
3,SJK121110505667,UQ269,2,6.2,6.2,22.5,6.2 x 6.2 x 22.5
4,SJK122011012218,UQ166,1,15.5,2.5,4.0,15.5 x 2.5 x 4
...,...,...,...,...,...,...,...
995,SJK121082304769,UQ205,1,4.0,4.0,13.0,4 x 4 x 13
996,SJK121102502007,UQ122,1,8.0,2.5,18.0,8 x 2.5 x 18
997,SJK122071903317,UQ166,1,15.5,2.5,4.0,15.5 x 2.5 x 4
998,SJK122030313686,UQ121,1,14.4,2.5,15.0,14.4 x 2.5 x 15


In [15]:
box_numbers = box_dimension_df['box_number'].values
xs = box_dimension_df['length'].values
ys = box_dimension_df['width'].values
zs = box_dimension_df['height'].values

def solve(order_df: pd.DataFrame):
    for box_number, x, y, z in zip(box_numbers, xs, ys, zs):
        packer = Packer()
        packer.add_bin(Bin(box_number, x, y, z, MAX_WEIGHT))

        for _, row in order_df.iterrows():
            for _ in range(row['quantity']):
                packer.add_item(Item(row['sku_id'], row['length'], row['width'], row['height'], 1))

        packer.pack()

        for b in packer.bins:
            if len(b.unfitted_items) == 0:
                return box_number
            
    return 'UNFITTED'

In [16]:
ddf = df.copy()

res_matrix = []
for order_number, order_df in tqdm(ddf.groupby('order_number')):
    res_matrix.append([order_number, solve(order_df)])
    
res_df = pd.DataFrame(data=res_matrix, columns=['order_number', 'box_number'])

100%|██████████| 998/998 [00:29<00:00, 33.83it/s]


In [17]:
## CLEAN FINAL DATAFRAME

to_basket_df = order_sku_df.copy()

to_basket_df['total_basket_sku'] = to_basket_df.groupby(['order_number'])['sku_id'].transform('nunique')
to_basket_df['total_basket_pcs'] = to_basket_df.groupby(['order_number'])['quantity'].transform('sum')

basket_df = to_basket_df[['order_number','total_basket_sku','total_basket_pcs']]
basket_df.columns = ['order_number','total_sku','total_qty']
basket_df = basket_df.drop_duplicates()

In [18]:
final_df = res_df.merge(box_dimension_df[['box_number','box_dimension']], how='left', on='box_number')
final_df = final_df.merge(basket_df, how='left', on='order_number')

final_df

,order_number,box_number,box_dimension,total_sku,total_qty
0,SJK121042307013,BOX-49,10.0 x 12.0 x 18.0,1,1
1,SJK121042307014,BOX-7,10.0 x 10.0 x 16.0,1,1
2,SJK121042402429,BOX-6,10.0 x 10.0 x 15.0,1,1
3,SJK121042703018,BOX-72,10.0 x 13.0 x 21.0,1,2
4,SJK121042907803,BOX-6,10.0 x 10.0 x 15.0,1,1
...,...,...,...,...,...
993,SSB122050600004,BOX-14,10.0 x 10.0 x 23.0,1,1
994,SSB122050900003,BOX-5,10.0 x 10.0 x 14.0,1,1
995,SSB122080500038,BOX-12,10.0 x 10.0 x 21.0,1,1
996,SSB122081500032,BOX-96,10.0 x 14.0 x 25.0,1,2


In [19]:
final_df = final_df.groupby(['box_dimension','box_number']).agg({'order_number' : ['count']}).reset_index()

final_df.columns = ['box_dimension','box_number','total_order']

final_df = final_df.sort_values(['total_order'],ascending=False)

In [20]:
final_df['all_order'] = final_df['total_order'].sum()
final_df['perc_of_order'] = round((final_df['total_order'] / final_df['all_order']) * 100,1)
final_df['perc_of_order'] = final_df['perc_of_order'].astype(str) + '%'

final_df.drop(['all_order'], inplace=True, axis=1)

final_df.head(5)

,box_dimension,box_number,total_order,perc_of_order
13,10.0 x 10.0 x 23.0,BOX-14,98,9.9%
11,10.0 x 10.0 x 21.0,BOX-12,73,7.4%
6,10.0 x 10.0 x 16.0,BOX-7,66,6.7%
5,10.0 x 10.0 x 15.0,BOX-6,63,6.4%
12,10.0 x 10.0 x 22.0,BOX-13,61,6.2%
